- 时间跨度
    - 历史数据中包含`('2017-01-01 00:00:00', '2018-03-27 05:00:00')`的天气数据
    - API 下载数据中包含 `'2017-04-01 00:00:00'` 以后的数据
    - 拟使用四月的数据作为 dev set，历史数据作为训练集

In [3]:
import numpy as np
import datetime
import pandas as pd
from utils.weather_data_util import load_bj_grid_meo_data

### 1. 首先加载使用到的 meo 数据

In [4]:
near_stations = {'aotizhongxin_aq': 'beijing_grid_304',
 'badaling_aq': 'beijing_grid_224',
 'beibuxinqu_aq': 'beijing_grid_263',
 'daxing_aq': 'beijing_grid_301',
 'dingling_aq': 'beijing_grid_265',
 'donggaocun_aq': 'beijing_grid_452',
 'dongsi_aq': 'beijing_grid_303',
 'dongsihuan_aq': 'beijing_grid_324',
 'fangshan_aq': 'beijing_grid_238',
 'fengtaihuayuan_aq': 'beijing_grid_282',
 'guanyuan_aq': 'beijing_grid_282',
 'gucheng_aq': 'beijing_grid_261',
 'huairou_aq': 'beijing_grid_349',
 'liulihe_aq': 'beijing_grid_216',
 'mentougou_aq': 'beijing_grid_240',
 'miyun_aq': 'beijing_grid_392',
 'miyunshuiku_aq': 'beijing_grid_414',
 'nansanhuan_aq': 'beijing_grid_303',
 'nongzhanguan_aq': 'beijing_grid_324',
 'pingchang_aq': 'beijing_grid_264',
 'pinggu_aq': 'beijing_grid_452',
 'qianmen_aq': 'beijing_grid_303',
 'shunyi_aq': 'beijing_grid_368',
 'tiantan_aq': 'beijing_grid_303',
 'tongzhou_aq': 'beijing_grid_366',
 'wanliu_aq': 'beijing_grid_283',
 'wanshouxigong_aq': 'beijing_grid_303',
 'xizhimenbei_aq': 'beijing_grid_283',
 'yanqin_aq': 'beijing_grid_225',
 'yizhuang_aq': 'beijing_grid_323',
 'yongdingmennei_aq': 'beijing_grid_303',
 'yongledian_aq': 'beijing_grid_385',
 'yufa_aq': 'beijing_grid_278',
 'yungang_aq': 'beijing_grid_239',
 'zhiwuyuan_aq': 'beijing_grid_262'}

In [5]:
# bj_meo_stations : meo_station as key and df as value 
bj_grid_meo_dataset, stations, bj_meo_stations = load_bj_grid_meo_data(near_stations)

In [7]:
bj_grid_meo_dataset.index.max()

Timestamp('2018-04-23 23:00:00')

In [4]:
bj_meo_stations['zhiwuyuan_aq'].head()

,zhiwuyuan_aq_temperature,zhiwuyuan_aq_pressure,zhiwuyuan_aq_humidity,zhiwuyuan_aq_wind_direction,zhiwuyuan_aq_wind_speed/kph
time,,,,,
2017-01-01 00:00:00,-6.01,1003.24,69.97,129.93,1.51
2017-01-01 01:00:00,-3.19,1003.12,58.66,147.40,1.37
2017-01-01 02:00:00,-0.37,1003.00,47.35,166.69,1.38
2017-01-01 03:00:00,2.45,1002.89,36.04,183.97,1.53
2017-01-01 04:00:00,3.55,1002.25,33.61,154.96,2.27


In [5]:
len(bj_meo_stations.keys())

35

### 2. 重复值分析

- 识别重复值的数量，并将重复值去掉
- meo 数据中没有重复数据

In [6]:
for station in bj_meo_stations.keys() :
    
    df = bj_meo_stations[station].copy()
    length = df.shape[0]
    order = range(length)
    df['order'] = pd.Series(order, index=df.index)    
    
    
    df["time"] = df.index
    df.set_index("order", inplace=True)
    
    length_1 = df.shape[0]
    # print("重复值去除之前，共有数据数量", df.shape[0])
    
    used_times = []
    for index in df.index :
        time = df.loc[index]["time"]
        if time not in used_times :
            used_times.append(time)
        else : 
            df.drop([index], inplace=True)

    length_2 = df.shape[0]
    delta = length_1 - length_2
    # print("重复值去除之后，共有数据数量", df.shape[0])
    print("%s 重复数量 : %d" %(station, delta))
    
    df.set_index("time", inplace=True)
    bj_meo_stations[station] = df

dongsi_aq 重复数量 : 0
liulihe_aq 重复数量 : 0
beibuxinqu_aq 重复数量 : 0
badaling_aq 重复数量 : 0
shunyi_aq 重复数量 : 0
yongdingmennei_aq 重复数量 : 0
fangshan_aq 重复数量 : 0
wanliu_aq 重复数量 : 0
qianmen_aq 重复数量 : 0
donggaocun_aq 重复数量 : 0
gucheng_aq 重复数量 : 0
nansanhuan_aq 重复数量 : 0
guanyuan_aq 重复数量 : 0
dongsihuan_aq 重复数量 : 0
nongzhanguan_aq 重复数量 : 0
yizhuang_aq 重复数量 : 0
yanqin_aq 重复数量 : 0
miyun_aq 重复数量 : 0
mentougou_aq 重复数量 : 0
yufa_aq 重复数量 : 0
aotizhongxin_aq 重复数量 : 0
wanshouxigong_aq 重复数量 : 0
zhiwuyuan_aq 重复数量 : 0
tiantan_aq 重复数量 : 0
huairou_aq 重复数量 : 0
daxing_aq 重复数量 : 0
miyunshuiku_aq 重复数量 : 0
xizhimenbei_aq 重复数量 : 0
tongzhou_aq 重复数量 : 0
yungang_aq 重复数量 : 0
dingling_aq 重复数量 : 0
fengtaihuayuan_aq 重复数量 : 0
pingchang_aq 重复数量 : 0
yongledian_aq 重复数量 : 0
pinggu_aq 重复数量 : 0


### 3. 缺失值分析

#### 3.1 局部缺失

判断有没有局部缺失：没有局部缺失！

In [7]:
for station in bj_meo_stations.keys():
    df = bj_meo_stations[station]
    print(station, pd.isnull(df).any().any())

dongsi_aq False
liulihe_aq False
beibuxinqu_aq False
badaling_aq False
shunyi_aq False
yongdingmennei_aq False
fangshan_aq False
wanliu_aq False
qianmen_aq False
donggaocun_aq False
gucheng_aq False
nansanhuan_aq False
guanyuan_aq False
dongsihuan_aq False
nongzhanguan_aq False
yizhuang_aq False
yanqin_aq False
miyun_aq False
mentougou_aq False
yufa_aq False
aotizhongxin_aq False
wanshouxigong_aq False
zhiwuyuan_aq False
tiantan_aq False
huairou_aq False
daxing_aq False
miyunshuiku_aq False
xizhimenbei_aq False
tongzhou_aq False
yungang_aq False
dingling_aq False
fengtaihuayuan_aq False
pingchang_aq False
yongledian_aq False
pinggu_aq False


#### 3.2 整体缺失

统计缺失小时的连续值
- 如果一个缺失小时在一个长度小于等于5小时的缺失时段内，就进行补全
- 如果超过5小时，就舍弃，将全部值补成 NAN，**这也是整个表中唯一可能出现 NAN 的情况**

In [9]:
for station in bj_meo_stations.keys() :
    df = bj_meo_stations[station].copy()
    
    min_time = df.index.min()
    max_time = df.index.max()

    min_time = datetime.datetime.strptime(min_time, '%Y-%m-%d %H:%M:%S')
    max_time = datetime.datetime.strptime(max_time, '%Y-%m-%d %H:%M:%S')
    delta_all = max_time - min_time
    
    all_length = delta_all.total_seconds()/3600 + 1
    real_length = df.shape[0]
    # print("在空气质量数据时间段内，总共应该有 %d 个小时节点。" %(all_length))
    # print("实际的时间节点数是 ", real_length)
    print("%s 缺失时间节点数量是 %d" %(station, all_length-real_length))

dongsi_aq 缺失时间节点数量是 121
liulihe_aq 缺失时间节点数量是 121
beibuxinqu_aq 缺失时间节点数量是 121
badaling_aq 缺失时间节点数量是 121
shunyi_aq 缺失时间节点数量是 121
yongdingmennei_aq 缺失时间节点数量是 121
fangshan_aq 缺失时间节点数量是 121
wanliu_aq 缺失时间节点数量是 121
qianmen_aq 缺失时间节点数量是 121
donggaocun_aq 缺失时间节点数量是 121
gucheng_aq 缺失时间节点数量是 121
nansanhuan_aq 缺失时间节点数量是 121
guanyuan_aq 缺失时间节点数量是 121
dongsihuan_aq 缺失时间节点数量是 121
nongzhanguan_aq 缺失时间节点数量是 121
yizhuang_aq 缺失时间节点数量是 121
yanqin_aq 缺失时间节点数量是 121
miyun_aq 缺失时间节点数量是 121
mentougou_aq 缺失时间节点数量是 122
yufa_aq 缺失时间节点数量是 121
aotizhongxin_aq 缺失时间节点数量是 122
wanshouxigong_aq 缺失时间节点数量是 121
zhiwuyuan_aq 缺失时间节点数量是 121
tiantan_aq 缺失时间节点数量是 121
huairou_aq 缺失时间节点数量是 121
daxing_aq 缺失时间节点数量是 121
miyunshuiku_aq 缺失时间节点数量是 121
xizhimenbei_aq 缺失时间节点数量是 121
tongzhou_aq 缺失时间节点数量是 121
yungang_aq 缺失时间节点数量是 121
dingling_aq 缺失时间节点数量是 121
fengtaihuayuan_aq 缺失时间节点数量是 121
pingchang_aq 缺失时间节点数量是 121
yongledian_aq 缺失时间节点数量是 121
pinggu_aq 缺失时间节点数量是 121


#### 3.3 整体缺失补充

In [10]:
for station in bj_meo_stations.keys() :
    df = bj_meo_stations[station].copy()
    print(station, df.shape)

dongsi_aq (11351, 5)
liulihe_aq (11351, 5)
beibuxinqu_aq (11351, 5)
badaling_aq (11351, 5)
shunyi_aq (11351, 5)
yongdingmennei_aq (11351, 5)
fangshan_aq (11351, 5)
wanliu_aq (11351, 5)
qianmen_aq (11351, 5)
donggaocun_aq (11351, 5)
gucheng_aq (11351, 5)
nansanhuan_aq (11351, 5)
guanyuan_aq (11351, 5)
dongsihuan_aq (11351, 5)
nongzhanguan_aq (11351, 5)
yizhuang_aq (11351, 5)
yanqin_aq (11351, 5)
miyun_aq (11351, 5)
mentougou_aq (11350, 5)
yufa_aq (11351, 5)
aotizhongxin_aq (11350, 5)
wanshouxigong_aq (11351, 5)
zhiwuyuan_aq (11351, 5)
tiantan_aq (11351, 5)
huairou_aq (11351, 5)
daxing_aq (11351, 5)
miyunshuiku_aq (11351, 5)
xizhimenbei_aq (11351, 5)
tongzhou_aq (11351, 5)
yungang_aq (11351, 5)
dingling_aq (11351, 5)
fengtaihuayuan_aq (11351, 5)
pingchang_aq (11351, 5)
yongledian_aq (11351, 5)
pinggu_aq (11351, 5)


In [18]:
delta = datetime.timedelta(hours=1)

for station in bj_meo_stations.keys() :
    df = bj_meo_stations[station].copy()
    nan_series = pd.Series({key:np.nan for key in df.columns})
    
    min_time = df.index.min()
    max_time = df.index.max()

    min_time = datetime.datetime.strptime(min_time, '%Y-%m-%d %H:%M:%S')
    max_time = datetime.datetime.strptime(max_time, '%Y-%m-%d %H:%M:%S')

    time = min_time
    
    while time <=  max_time :
        
        time_str = datetime.date.strftime(time, '%Y-%m-%d %H:%M:%S')
        if time_str not in df.index :
            
            # 前边第几个是非空的
            found_for = False
            i = 0
            while not found_for :
                i += 1
                for_time = time - i * delta
                for_time_str = datetime.date.strftime(for_time, '%Y-%m-%d %H:%M:%S')
                if for_time_str in df.index :
                    for_row = df.loc[for_time_str]
                    for_step = i
                    found_for = True

            # 后边第几个是非空的
            found_back = False
            j = 0
            while not found_back :
                j += 1
                back_time = time + j * delta
                back_time_str = datetime.date.strftime(back_time, '%Y-%m-%d %H:%M:%S')
                if back_time_str in df.index :
                    back_row = df.loc[back_time_str]
                    back_step = j
                    found_back = True
        
            all_steps = for_step + back_step
        
            if all_steps <= 5 :
                delata_values = back_row - for_row
                df.loc[time_str] = for_row + (for_step/all_steps) * delata_values
            else :
                df.loc[time_str] = nan_series
        
        time += delta
    bj_meo_stations[station] = df
    
    print("%s : length of data is %d" %(station, df.shape[0]))

dongsi_aq : length of data is 11472
liulihe_aq : length of data is 11472
beibuxinqu_aq : length of data is 11472
badaling_aq : length of data is 11472
shunyi_aq : length of data is 11472
yongdingmennei_aq : length of data is 11472
fangshan_aq : length of data is 11472
wanliu_aq : length of data is 11472
qianmen_aq : length of data is 11472
donggaocun_aq : length of data is 11472
gucheng_aq : length of data is 11472
nansanhuan_aq : length of data is 11472
guanyuan_aq : length of data is 11472
dongsihuan_aq : length of data is 11472
nongzhanguan_aq : length of data is 11472
yizhuang_aq : length of data is 11472
yanqin_aq : length of data is 11472
miyun_aq : length of data is 11472
mentougou_aq : length of data is 11472
yufa_aq : length of data is 11472
aotizhongxin_aq : length of data is 11472
wanshouxigong_aq : length of data is 11472
zhiwuyuan_aq : length of data is 11472
tiantan_aq : length of data is 11472
huairou_aq : length of data is 11472
daxing_aq : length of data is 11472
miyun

#### 3.4 风向缺失值处理
- 暂时使用 0 替换缺失的风向

In [19]:
for station in bj_meo_stations.keys():
    df = bj_meo_stations[station].copy()
    df.replace(999017,0, inplace=True)
    bj_meo_stations[station] = df

#### 3.5 拼成整表，并保存

In [20]:
bj_meo_stations_merged = pd.concat(list(bj_meo_stations.values()), axis=1)

In [21]:
bj_meo_stations_merged.shape

(11472, 175)

In [22]:
bj_meo_stations_merged.sort_index(inplace=True)

In [23]:
bj_meo_stations_merged.to_csv("data/bj_meo_data.csv")

需要将 `bj_meo_data.csv`左上角位置的空格补上 "time" 

### 4 数据归一化

In [26]:
describe = bj_meo_stations_merged.describe()
describe.to_csv("data/bj_meo_describe.csv")

In [27]:
df_norm = (bj_meo_stations_merged - describe.loc['mean']) / describe.loc['std']
df_norm.to_csv("data/bj_meo_norm_data.csv")

### Changelog
- 0425 v0
    - 天气数据的缺失值补全
    - 将数据保存到 `data/bj_meo_data.csv`中
- 0426 v0.1
    - 增加数据正则化